# 3.3 Implementation of EM algorithm [17 pts]

3.3.1. Requirements and suggestions

3.3.2. Some questions

In [2]:
import numpy as np 
import scipy.io as sio
import matplotlib.pyplot as plt
import random

In [3]:
data = sio.loadmat("q3_data.mat")
print("dd")

dd


In [4]:
data.keys()

dict_keys(['__globals__', '__header__', '__version__', 'Y', 'X'])

In [5]:
def train_test_split(X, Y, rate):
    num_sample = X.shape[0]

    test_index_set = set(random.sample(range(num_sample), int(num_sample*rate)))
    total_index_set = set(range(num_sample))

    train_index_set = total_index_set - test_index_set
    train_index = list(train_index_set)
    test_index = list(test_index_set)

    train_x = X[train_index,:]
    train_y = Y[train_index, :]

    test_x = X[test_index,:]
    test_y = Y[test_index,:]

    return train_x, train_y, test_x, test_y


In [9]:
X = data['X']
Y = data['Y']
print(X)
print(X.shape)
print(Y)
print(Y.shape)
train_x, train_y, test_x, test_y = train_test_split(X, Y, 0.4)

[[-4.71238898]
 [-4.70767423]
 [-4.70295949]
 ..., 
 [ 4.70295949]
 [ 4.70767423]
 [ 4.71238898]]
(2000, 1)
[[ 0.41220109]
 [ 2.25572172]
 [ 0.32923398]
 ..., 
 [-0.11020524]
 [-0.14431699]
 [ 0.29909876]]
(2000, 1)


## 3.3.1.  Requirements and suggestions

In this question, we are going to implement EM algorithm for HME model, following the updating rules derived in part 3.2. Here are some requirements and suggestions for your implementation:

### Pad input $\mathbf{X}$ with $1$

To allow bias terms in the model, you should pad $\mathbf{X}$ with a column with value 1. 

### IRLS algorithm
If you find IRLS algorithm update not stable, i.e. generate singular matrix in the update, you're also allowed to directly use Newton's method, as the IRLS algorithm here is a special case of Newton's method. 

### Parameter initialization
You may explore different ways of parameter initialization, as a suggestion, initialization with uniform distribution Unif(0, 1) for all parameters is fine. 

### Parameters in the algorithm
Set the max iteration as 50 for the main loop. 

For IRLS algorithm, set max iteration as 5, learning rate $\lambda$ as 1, using a stopping criterion 
$$|f^{k+1} - f^{k}| < 1e-6$$
where $f$ is the objective function. 

### The Log Sum Exp trick 

To reduce numerical error and prevent underflows, you may consider use log-sum-exp trick for the computing of some variables. 

### Clip some variables
In the updating, due to numerical error, some variables, i.e. probabilities, may be very close to 0 or 1, which may make some matrices singular. To avoid this effect, we may need to clip these variables. You may clip them to $[1E{-10}, 1-1E{-10}]$. 

Hint: you may consider to clip $g_i^{(t)}$, $g_{j|i}^{(t)}$ and $\mathbb{E}[z^{(t)}_{ij}|\mathcal{X}]$. 

### Monitor log-likelihood

It is good to monitor the log-likelihood in each iteration, as in EM algorithm, the log-likelihood should keep increasing, which could be a good check of whether the implementation is right. 

### Program Language

You may use either Python or MATLAB in your implementation, as long as you can configure either Python or MATLAB for Jupyter notebook. 

In [7]:
# Implemente your algorithm here

class IRLS(object):

    def __init__(self, threshold, max_iter):
        self.threshold = threshold
        self.theta = None
        self.var = 0
        self.delta_param_norm = 0
        self.deta_log_like = 0
        self.iter = max_iter

    def init_param(self, x_dim):
        self.theta = np.random.normal(0,1,m)
        self.var = 1

    def fit(self, X, Y, weights = None):

        n,m = X.shape

        if weights == None:
            w = np.ones(n)
            weights = w
        else:
            w = np.sqrt(weights)

        X_w = np.transpose(np.transpose(X) * w)
        Y_w = Y*w

        if self.theta == None:
            self.init_params(m)
        theta_tmp = self.theta
        var_tmp = self.var 
        log_likelihood_tmp = self.calc_log_likelihood(X, Y, weights)

        self.theta, _, _, _ = LA.lstsq(X, Y)

        pred_diff = (Y_w - np.dot(X_w,self.theta))
        pred_diff_variance = np.dot(pred_diff,pred_diff)/np.sum(weights)

        # update variance
        self.var = pred_diff_variance
        # recalculate parameters
        log_like_after = self.calc_log_likelihood(X,Y,weights)
        delta_log_like = ( log_like_after - log_like_before)/n

        # undo the update variance if underflow
        if delta_log_like < self.stop_learning:
            self.theta = theta_recovery
            self.var   = var_recovery
            delta_log_like = 0

        # otherwise save change in parameters and likelihood
        theta_delta = self.theta - theta_tmp
        self.delta_param_norm = np.sum(np.dot(theta_delta.T,theta_delta))
        self.delta_log_like = delta_log_like

    def predict(self, X):
        return np.dot(X, self.theta)

    def calc_post_log_prob(self, x, y):
        if self.thsta == None:
            self.init_params(x.shape[1])
        # return normal pdf, log pdf
        # assume Gaussian
        u = y - np.dot(x,self.theta)
        log_norm = -1* np.log(np.sqrt(2*np.pi*self.var))
        log_main = -u*u/(2*self.var)
        log_pdf = log_norm + log_main
        prob = np.exp(log_pdf)
        return (log_pdf,prob)

    def calc_log_likelihood(self, x, y, weights = None):
        if weights == None:
            weights = np.ones(x,shape[0])
        log_pdf, prob = self.calc_post_log_prob(x, y)
        loglikelihood = np.sum(weights * log_pdf)

        return log_likelihood

## 3.3.2. Some questions

Using the provided data ***Question_3.3-data.mat*** as input for your implementation. 

1. Use M = 2, N = 2 to run your algorithm, report RMSE for the prediction, plot log-likelihood as a function of iteration, plot output $y$ and prediction $\hat y$ as a function of $x$ in the same plot, and visualize $\mathbb{E}[z^{(1)}_{ij}|\mathcal{X}]$, $\mathbb{E}[z^{(1000)}_{ij}|\mathcal{X}]$, $i=1:M~~j=1:N$ as heat maps. 

2. Use M = 5, N = 5 to run your algorithm, report RMSE for the prediction, plot log-likelihood as a function of iteration, plot output $y$ and prediction $\hat y$ as a function of $x$ in the same plot, and visualize $\mathbb{E}[z^{(1)}_{ij}|\mathcal{X}]$, $\mathbb{E}[z^{(1000)}_{ij}|\mathcal{X}]$, $i=1:M~~j=1:N$ as heat maps. 

3. What information could you get form heat maps? i.e. the number of activated expert networks. What's the difference between the two settings in terms of log-likelihood and prediction?


In [8]:
hme = HME(train_x, train_y, 2, 2, max_iter = 100)


NameError: name 'HME' is not defined